In [ ]:
pip install tensorflow==1.14.0

     |████████████████████████████████| 109.3 MB 1.4 MB/s 
     |████████████████████████████████| 3.1 MB 38.5 MB/s 
     |████████████████████████████████| 488 kB 39.2 MB/s 
     |████████████████████████████████| 50 kB 7.1 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.6.0
    Uninstalling tensorflow-estimator-2.6.0:
      Successfully uninstalled tensorflow-estimator-2.6.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.6.0
    Uninstalling tensorboard-2.6.0:
      Successfully uninstalled tensorboard-2.6.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.6.0
    Uninstalling tensorflow-2.6.0:
      Successfully uninstalled tensorflow-2.6.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kapre 0.3.5 requires tensorflow>=2.0.0, but yo

In [ ]:
!pip install keras==2.2.4

     |████████████████████████████████| 312 kB 12.6 MB/s 
  Attempting uninstall: keras
    Found existing installation: keras 2.6.0
    Uninstalling keras-2.6.0:
      Successfully uninstalled keras-2.6.0


In [3]:
%tensorflow_version 1.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

TensorFlow 1.x selected.
Found GPU at: /device:GPU:0


In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

import numpy as np
np.random.seed(7)

import matplotlib
matplotlib.use("Agg")

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam
from tensorflow.python.framework import graph_io
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import tensorflow as tf
import pickle
import os

Mounted at /content/gdrive


In [5]:

def plot_loss(H, N, plotPath):
  # construct a plot that plots and saves the training history
	plt.style.use("ggplot")
	plt.figure()
	plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
	plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
	plt.title("Syringe Training Loss")
 
	plt.xlabel("Epoch #")
	plt.ylabel("Loss")
	plt.legend(loc="upper right")
	plt.savefig(plotPath)

def plot_accuracy(H, N, plotPath):
  # construct a plot that plots and saves the training history
  plt.style.use("ggplot")
  plt.figure()
  plt.plot(np.arange(0, N), H.history["acc"], label="train_acc")
  plt.plot(np.arange(0, N), H.history["val_acc"], label="val_acc")
  plt.title("Syringe Training Accuracy")

  plt.xlabel("Epoch #")
  plt.ylabel("Accuracy")
  plt.legend(loc="lower right")
  plt.savefig(plotPath)
   
conf = {
    "base_path": "/content/gdrive/MyDrive/Colab Notebooks/dataset",
    "pb": "/content/gdrive/MyDrive/Colab Notebooks/model",
    "plot_loss":"/content/gdrive/MyDrive/Colab Notebooks/output/plot_loss.png",
    "plot_accuracy":"/content/gdrive/MyDrive/Colab Notebooks/output/plot_accuracy.png"
}

# define the constants
BATCH_SIZE = 64
LEARNING_RATE = 0.0002
EPOCHS = 64

CLASS_NAMES = ["dirty", "clean","wet"]
	

# grab the path to our data created using dataset building script
basePath = conf["base_path"]

# build the paths to the data files
trainXPath = os.path.join(basePath, "trainX.cpickle")
testXPath = os.path.join(basePath, "testX.cpickle")
trainYPath = os.path.join(basePath, "trainY.cpickle")
testYPath = os.path.join(basePath, "testY.cpickle")

# load the data files 
print("[INFO] loading data files...")
trainX = pickle.loads(open(trainXPath, "rb").read())
trainY = pickle.loads(open(trainYPath, "rb").read())
testX = pickle.loads(open(testXPath, "rb").read())
testY = pickle.loads(open(testYPath, "rb").read())

print(len(trainX))
print(len(trainY))
print(len(testX))
print(len(testY))

# account for skew in the labeled data
classTotals = trainY.sum(axis=0)
classWeight = classTotals.max() / classTotals

# initialize the training data augmentation object
# trainAug = ImageDataGenerator( horizontal_flip=True, fill_mode="nearest")
trainAug = ImageDataGenerator( rotation_range=30, width_shift_range=0.1,
                              height_shift_range=0.1, shear_range=0.2,
                              zoom_range=0.2, horizontal_flip=True, 
                              vertical_flip=True, fill_mode="nearest")
 
# initialize the validation/testing data augmentation object (which
# we'll be adding mean subtraction to)
valAug = ImageDataGenerator()
 
# define the ImageNet mean subtraction (in RGB order) and set the
# the mean subtraction value for each of the data augmentation
# objects
mean = np.array([123.68, 116.779, 103.939], dtype="float32")
trainAug.mean = mean
valAug.mean = mean

# initialize the training generator
trainGen = trainAug.flow(
	trainX,
	trainY,
	shuffle=True,
	batch_size=BATCH_SIZE,
	seed=42)
 
# initialize the testing generator
testGen = valAug.flow(
	testX,
	testY,
	shuffle=False,
	batch_size=BATCH_SIZE,
	seed=42)

# load VGG16 without the head layers
baseModel = VGG16(weights="imagenet", include_top=False, input_tensor=Input(shape=(224, 224, 3)))
 
# construct the head of the model that will be placed on top of the
# the base model
headModel = baseModel.output
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(512, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(len(CLASS_NAMES), activation="softmax")(headModel)
 
# place the head FC model on top of the base model (this will become
# the actual model we will train)
model = Model(inputs=baseModel.input, outputs=headModel)

# loop over all layers in the base model and freeze them so they will
# *not* be updated during the first training process
for layer in baseModel.layers:
	layer.trainable = False

# compile our model (this needs to be done after our setting our
# layers to being non-trainable
print("[INFO] compiling model...")
opt = Adam(lr=LEARNING_RATE)
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
 
# train the head of the network for a few epochs (all other layers
# are frozen) -- this will allow the new FC layers to start to become
# initialized with actual "learned" values versus pure random
print("[INFO] training head...")
H = model.fit_generator(trainGen, 
                        steps_per_epoch=trainX.shape[0] // BATCH_SIZE,
                        validation_data=testGen, 
                        validation_steps=testX.shape[0] // BATCH_SIZE, 
                        class_weight=classWeight, 
                        epochs=EPOCHS)

# reset the testing generator and evaluate the network after
# fine-tuning just the network head
print("[INFO] evaluating after fine-tuning network head...")
testGen.reset()
predIdxs = model.predict(testGen, steps=(testX.shape[0] // BATCH_SIZE) + 1)
predIdxs = np.argmax(predIdxs, axis=1)
print(len(predIdxs))

#print the classification report 
print(classification_report(np.argmax(testY, axis=1), predIdxs, target_names=CLASS_NAMES))
plot_loss(H, EPOCHS, conf["plot_loss"])
plot_accuracy(H, EPOCHS, conf["plot_accuracy"])

# serialize the model as a native tf graph to disk
print("[INFO] serializing network to a native tensorflow graph...")
sess = K.get_session()
frozen = tf.graph_util.convert_variables_to_constants(sess, sess.graph_def, [model.output.op.name])
graph_io.write_graph(frozen, conf["pb"], "syringe.pb", as_text=False)


[INFO] loading data files...
3349
3349
838
838
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
58892288/58889256 [==============================] - 1s 0us/step
[INFO] compiling model...
[INFO] training head...
Epoch 1/64
52/52 [==============================] - 60s 1s/step - loss: 6.6647 - acc: 0.5546 - val_loss: 1.3396 - val_acc: 0.7320
Epoch 2/64
52/52 [==============================] - 52s 999ms/step - loss: 1.1542 - acc: 0.6307 - val_loss: 0.7024 - val_acc: 0.7428
Epoch 3/64
52/52 [==============================] - 51s 983ms/step - loss: 0.8476 - acc: 0.6612 - val_loss: 0.6631 - val_acc: 0.7584
Epoch 4/64
52/52 [==============================] - 51s 984ms/step - loss: 0.7647 - acc: 0.7026 - val_loss: 0.6627 - val_acc: 0.7716
Epoch 5/64
52/52 [==============================] - 51s 986ms/step - loss: 0.6771 - acc: 0.7312 - val_loss: 0.5504 - val_acc: 0.7957
Epoch 6/64
52/52 [==============================] - 51s 972ms/step - loss: 0.6205 - acc: 0.7479

'/content/gdrive/MyDrive/Colab Notebooks/model/syringe.pb'

In [ ]:
model.save('/content/gdrive/MyDrive/Colab Notebooks/model/mySyringe.h5')
model.save('/content/gdrive/MyDrive/Colab Notebooks/model/mySyringe.pb')

In [15]:
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

y_test = np.argmax(testY, axis=1)
cm = confusion_matrix(y_test, predIdxs)

disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=CLASS_NAMES)
disp.plot(cmap=plt.cm.Blues)
plt.show()